In [1]:
import torchreid

/home/appuser/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


# Data Prepareation

In [2]:
# download function は機能していなかったので、手動で deep-person-reid/reid-data/market1501/ の中にデータを設置した
datamanager = torchreid.data.ImageDataManager(
    root="../reid-data",
    sources="market1501",
    targets="market1501",
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/home/appuser/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/home/appuser/reid-data/market1501/Market-1501-v15.09.15.zip"


URLError: <urlopen error [Errno 110] Connection timed out>

# Model Preparation

In [3]:
model = torchreid.models.build_model(
    name="resnet50",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [4]:
# 過去に学習した チェックポイント (学習済みパラメータ) を再度利用する
from torchreid.utils import resume_from_checkpoint


# model ファイルのパス
fpath = '../log/resnet50/model/model.pth.tar-60'
start_epoch = resume_from_checkpoint(
    fpath=fpath,
    model=model, optimizer=optimizer, scheduler=scheduler
)

Loading checkpoint from "../log/resnet50/model/model.pth.tar-60"
Loaded model weights
Loaded optimizer
Loaded scheduler
Last epoch = 60
Last rank1 = 85.6%


In [5]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

# Training

- 実行は一回で良い
- `save_dir`には、学習結果が保存される

In [ ]:
# engine.run(
#     save_dir="log/resnet50",
#     max_epoch=60,
#     eval_freq=10,
#     print_freq=10,
#     test_only=False
# )


# Visualize

In [6]:
engine.run(
    save_dir="log/resnet50",
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=True,
    visrank=True
)

##### Evaluating market1501 (source) #####
Extracting features from query set ...
Done, obtained 3369-by-2048 matrix
Extracting features from gallery set ...
Done, obtained 15914-by-2048 matrix
Speed: 0.0193 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 69.3%
CMC curve
Rank-1  : 85.6%
Rank-5  : 93.7%
Rank-10 : 95.8%
Rank-20 : 97.6%
# query: 3369
# gallery 15914
Visualizing top-10 ranks ...
- done 100/3369
- done 200/3369
- done 300/3369
- done 400/3369
- done 500/3369
- done 600/3369
- done 700/3369
- done 800/3369
- done 900/3369
- done 1000/3369
- done 1100/3369
- done 1200/3369
- done 1300/3369
- done 1400/3369
- done 1500/3369
- done 1600/3369
- done 1700/3369
- done 1800/3369
- done 1900/3369
- done 2000/3369
- done 2100/3369
- done 2200/3369
- done 2300/3369
- done 2400/3369
- done 2500/3369
- done 2600/3369
- done 2700/3369
- done 2800/3369
- done 2900/3369
- done 3000/3369
- done 3100/3369
- done 3200/3369
- done 3300